In [20]:
import pysal as ps
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

from math import ceil

from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler, MinMaxScaler

rbs = RobustScaler(quantile_range=[0.025,0.975])
mms = MinMaxScaler(feature_range=(0,1))
pts = PowerTransformer()
scaler = StandardScaler()

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, DBSCAN, OPTICS
from esda.adbscan import ADBSCAN
from sklearn.decomposition import PCA

pd.get_option("max_info_columns")

300

In [21]:
pd.set_option("display.max_info_columns", 300) 

url = 'https://github.com/Lingweiz1998/DSSS_final_2020/raw/main/Data/LSOA.geojson'
gdf = gpd.read_file(url)

gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4835 entries, 0 to 4834
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   LSOA11CD   4835 non-null   object  
 1   LSOA11NM   4835 non-null   object  
 2   MSOA11CD   4835 non-null   object  
 3   MSOA11NM   4835 non-null   object  
 4   LAD11CD    4835 non-null   object  
 5   LAD11NM    4835 non-null   object  
 6   RGN11CD    4835 non-null   object  
 7   RGN11NM    4835 non-null   object  
 8   USUALRES   4835 non-null   int64   
 9   HHOLDRES   4835 non-null   int64   
 10  COMESTRES  4835 non-null   int64   
 11  POPDEN     4835 non-null   float64 
 12  HHOLDS     4835 non-null   int64   
 13  AVHHOLDSZ  4835 non-null   float64 
 14  geometry   4835 non-null   geometry
dtypes: float64(2), geometry(1), int64(4), object(8)
memory usage: 566.7+ KB


In [22]:
gdf.head()

,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,USUALRES,HHOLDRES,COMESTRES,POPDEN,HHOLDS,AVHHOLDSZ,geometry
0,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London,1465,1465,0,112.9,876,1.7,"MULTIPOLYGON (((532105.092 182011.230, 532162...."
1,E01000002,City of London 001B,E02000001,City of London 001,E09000001,City of London,E12000007,London,1436,1436,0,62.9,830,1.7,"MULTIPOLYGON (((532746.813 181786.891, 532671...."
2,E01000003,City of London 001C,E02000001,City of London 001,E09000001,City of London,E12000007,London,1346,1250,96,227.7,817,1.5,"MULTIPOLYGON (((532135.145 182198.119, 532158...."
3,E01000005,City of London 001E,E02000001,City of London 001,E09000001,City of London,E12000007,London,985,985,0,52.0,467,2.1,"MULTIPOLYGON (((533807.946 180767.770, 533649...."
4,E01000006,Barking and Dagenham 016A,E02000017,Barking and Dagenham 016,E09000002,Barking and Dagenham,E12000007,London,1703,1699,4,116.2,543,3.1,"MULTIPOLYGON (((545122.049 184314.931, 545271...."


In [3]:
df = pd.read_csv('https://data.london.gov.uk/download/lsoa-atlas/0193f884-2ccd-49c2-968e-28aa3b1c480d/lsoa-data.csv',
                 encoding= 'unicode_escape', na_values=[':'], low_memory=False)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4837 entries, 0 to 4836
Data columns (total 277 columns):
 #   Column                                                                                                            Non-Null Count  Dtype  
---  ------                                                                                                            --------------  -----  
 0   Lower Super Output Area                                                                                           4835 non-null   object 
 1   Names                                                                                                             4835 non-null   object 
 2   Mid-year Population Estimates;All Ages;2001                                                                       4835 non-null   float64
 3   Mid-year Population Estimates;All Ages;2002                                                                       4835 non-null   float64
 4   Mid-year Population Estimates

In [5]:
df = df[['Lower Super Output Area','Economic Activity;Economically active: Total;2011','Economic Activity;Economically inactive: Total;2011',
         'Economic Activity;Employment Rate;2011','Economic Activity;Unemployment Rate;2011','Household Income, 2011/12;Mean Annual Household Income estimate (£)',
         'Public Transport Accessibility Levels (2014);Average Score;','Public Transport Accessibility Levels (2014);% 4-6 (good access)','Public Transport Accessibility Levels (2014);% 0-1 (poor access)']]
df.head()

,Lower Super Output Area,Economic Activity;Economically active: Total;2011,Economic Activity;Economically inactive: Total;2011,Economic Activity;Employment Rate;2011,Economic Activity;Unemployment Rate;2011,"Household Income, 2011/12;Mean Annual Household Income estimate (£)",Public Transport Accessibility Levels (2014);Average Score;,Public Transport Accessibility Levels (2014);% 4-6 (good access),Public Transport Accessibility Levels (2014);% 0-1 (poor access)
0,E01000907,719.0,343.0,58.6,9.9,39985.0,4.6,45.9,0.0
1,E01000908,833.0,351.0,62.3,8.0,42394.0,2.9,0.0,33.7
2,E01000909,886.0,367.0,65.7,4.9,77370.0,3.3,0.0,0.0
3,E01000912,848.0,387.0,58.8,8.8,41737.0,4.6,41.6,0.0
4,E01000913,963.0,323.0,71.8,2.1,96649.0,2.8,0.5,27.8


In [6]:
df.columns = ['lsoa','Eco_Active','Eco_inact','Employment','Unemployment','Income_Mean','PTAL_Mean','PTAL_good','PTAL_bad']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4837 entries, 0 to 4836
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lsoa          4835 non-null   object 
 1   Eco_Active    4835 non-null   float64
 2   Eco_inact     4835 non-null   float64
 3   Employment    4835 non-null   float64
 4   Unemployment  4835 non-null   float64
 5   Income_Mean   4835 non-null   float64
 6   PTAL_Mean     4835 non-null   float64
 7   PTAL_good     4835 non-null   float64
 8   PTAL_bad      4835 non-null   float64
dtypes: float64(8), object(1)
memory usage: 340.2+ KB


In [7]:
pd.isnull(df).sum() > 0

lsoa            True
Eco_Active      True
Eco_inact       True
Employment      True
Unemployment    True
Income_Mean     True
PTAL_Mean       True
PTAL_good       True
PTAL_bad        True
dtype: bool

In [8]:
print(df[df['Eco_Active'].isnull()])

     lsoa  Eco_Active  Eco_inact  Employment  Unemployment  Income_Mean  \
4835  NaN         NaN        NaN         NaN           NaN          NaN   
4836  NaN         NaN        NaN         NaN           NaN          NaN   

      PTAL_Mean  PTAL_good  PTAL_bad  
4835        NaN        NaN       NaN  
4836        NaN        NaN       NaN  


In [9]:
print(df[df['PTAL_Mean'].isnull()])

     lsoa  Eco_Active  Eco_inact  Employment  Unemployment  Income_Mean  \
4835  NaN         NaN        NaN         NaN           NaN          NaN   
4836  NaN         NaN        NaN         NaN           NaN          NaN   

      PTAL_Mean  PTAL_good  PTAL_bad  
4835        NaN        NaN       NaN  
4836        NaN        NaN       NaN  


In [10]:
print(df.shape)

df = df.dropna()

print(df.shape)

(4837, 9)
(4835, 9)


In [11]:
ppd = pd.merge(gdf, df, left_on='LSOA11CD', right_on='lsoa', how='inner').reset_index()
ppd.shape
ppd.sample(3)

,index,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,USUALRES,...,geometry,lsoa,Eco_Active,Eco_inact,Employment,Unemployment,Income_Mean,PTAL_Mean,PTAL_good,PTAL_bad
4811,4811,E01033713,Hackney 010G,E02000354,Hackney 010,E09000012,Hackney,E12000007,London,1346,...,"MULTIPOLYGON (((534918.108 185913.311, 534885....",E01033713,799.0,275.0,62.8,11.8,36756.0,5.3,80.2,0.0
2645,2645,E01002707,Islington 022B,E02000575,Islington 022,E09000019,Islington,E12000007,London,1726,...,"MULTIPOLYGON (((532072.970 182916.481, 531978....",E01002707,1050.0,432.0,62.6,6.5,49015.0,6.6,100.0,0.0
1230,1230,E01001249,Ealing 028A,E02000265,Ealing 028,E09000009,Ealing,E12000007,London,2370,...,"MULTIPOLYGON (((516709.600 180490.567, 516715....",E01001249,1276.0,500.0,61.9,9.0,34340.0,5.1,77.8,0.0


In [12]:
def replace_legend_items(legend, mapping):
    for txt in legend.texts:
        for k,v in mapping.items():
            if txt.get_text() == str(k):
                txt.set_text(v)

In [13]:
tfont = {'fontname':'Liberation Sans Narrow', 'horizontalalignment':'left'}

In [14]:
f = plt.figure()
f.set_size_inches(12,7)

ax1 = f.add_subplot()
ppd.plot(column='PTAL_good', legend=True, cmap='plasma', figsize=(12,6), ax=ax1)

f.subplots_adjust(top=0.92)
f.suptitle(f"Acess to PT", x=0.025, size=24, **tfont);

ValueError: 'box_aspect' and 'fig_aspect' must be positive

ValueError: 'box_aspect' and 'fig_aspect' must be positive

<Figure size 864x504 with 2 Axes>

In [ ]:
ppd = ppd[['LSOA11NM','LSOA11CD','geometry','Eco_Active','Eco_inact','Employment','Unemployment','Income_Mean','PTAL_Mean','PTAL_good','PTAL_bad']].copy()
ppd.head()

In [ ]:
to_cluster = ppd[['LSOA11CD','Eco_Active','Eco_inact','Employment','Unemployment','Income_Mean','PTAL_Mean','PTAL_good','PTAL_bad']].set_index('LSOA11CD').copy()
to_cluster.head()

In [ ]:
plot_norm = sns.pairplot(to_cluster,height = 3)

In [ ]:
normed = to_cluster.copy()
for c in to_cluster.columns.values:
    normed[c] = pts.fit_transform(to_cluster[c].values.reshape(-1,1))
normed.sample(5)

In [ ]:
plot_norm = sns.pairplot(normed,height = 3)

In [ ]:
rand_st_int = 10 
pca = PCA(random_state=rand_st_int)

normed_pca = pca.fit_transform(normed)
print('Explained variance ratio of each component:')
print(pca.explained_variance_ratio_)

In [ ]:
dfpc = pd.DataFrame({'eigenvalue':pca.explained_variance_,
             'PC':list(range(1, pca.n_components_ + 1))})

plt.plot(dfpc['PC'], dfpc['eigenvalue'], marker='o', linestyle = '--')
plt.xlabel('Number of clusters')
plt.ylabel('Eigenvalue')
plt.show()

In [ ]:
pca = PCA(n_components = 3)

normed_pca = pca.fit_transform(normed)
print('Explained variance ratio of each component:')
print(pca.explained_variance_ratio_)

In [ ]:
list_var = normed.columns.tolist()
print(list_var)

In [ ]:
df_PrincipleComp = pd.DataFrame(pca.components_, columns = list_var)
df_PrincipleComp

In [ ]:
score_pca = pca.transform(normed)
score_pca

In [ ]:
print('Eigenvalues of each component:')
print(pca.explained_variance_)

In [ ]:
# calculate SSE for a range of number of cluster
list_SSE = []
min_k = 1
max_k = 10
range_k = range(min_k, max_k+1)
for i in range_k:
    km = KMeans(
        n_clusters=i, init='k-means++', random_state=0)
    km.fit(score_pca)
    # inertia is a concept in physics. Roughly it means SSE of clustering.
    list_SSE.append(km.inertia_)

# plot
plt.plot(range_k, list_SSE, marker='o', linestyle = '--')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

In [ ]:
k_cluster = 3
random_seed = 1
kmeans_method = KMeans(n_clusters=k_cluster,random_state=random_seed)
kmeans_method.fit(score_pca)

In [ ]:
ppd['Kmeans_Cluster'] = kmeans_method.labels_
ppd.plot(column='Kmeans_Cluster', categorical=True, legend=True, figsize=(12,8));

In [ ]:
ppd_kpca = pd.concat([ppd.reset_index(drop = True), pd.DataFrame(score_pca)],axis = 1)
ppd_kpca.columns.values[-3: ] = ['Component1','Component2','Component3']

In [ ]:
ppd_kpca['Segement'] = ppd_kpca['Kmeans_Cluster'].map({0:'First',1:'Second',2:'Third'})

In [ ]:
ppd_kpca.head()

In [ ]:
x_axis = ppd_kpca['Component1']
y_axis = ppd_kpca['Component2']
plt.figure(figsize = (10, 8))
sns.scatterplot(x_axis,y_axis,hue = ppd_kpca['Segement'], palette = ['g','r','c'])
plt.title('Clusters by PCA Components')
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
# axes instance
fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig)

# get colormap from seaborn
cmap = ListedColormap(sns.color_palette("RdYlBu_r", 256).as_hex())

x = ppd_kpca['Component1']
y = ppd_kpca['Component2']
z = ppd_kpca['Component3']

# plot
sc = ax.scatter(x, y, z, s=40, c=ppd_kpca['Kmeans_Cluster'], marker='.', cmap=cmap, alpha=1)
ax.set_xlabel('Component1')
ax.set_ylabel('Component2')
ax.set_zlabel('Component3')

# legend
plt.legend(*sc.legend_elements(), bbox_to_anchor=(1.05, 1), loc=2)

In [ ]:
plot_norm = sns.pairplot(ppd,hue = 'Kmeans_Cluster',height = 2,palette = 'Set1')